# Processamento na GPU [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/chcomin/curso-visao-computacional-2024/blob/main/M06_classificacao_de_imagens_naturais/3%20-%20Processamento%20na%20GPU.ipynb)

### Copiando dados entre CPU e GPU

In [1]:
import torch
from torch import nn
from torchvision import models

x = torch.rand(16,3,224,224)
model = models.resnet18()
print(x.device)
print(model.conv1.weight.device)

cpu
cpu


In [2]:
# Move o tensor para a GPU
x_cuda = x.to('cuda')
# Move todos os tensores de parâmetros do modelo para a GPU. Note que como `modelo`
# é um objeto, os atributos dele é que são modificados. A referência para o modelo na
# CPU é perdida
model.to('cuda')

print(x.device)
print(x_cuda.device)
print(model.conv1.weight.device)

cpu
cuda:0
cuda:0


Com os dados e o modelo na GPU, é possível aplicar o modelo normalmente:

In [3]:
y = model(x_cuda)

Portanto, um loop de treinamento padrão é feito da seguinte forma:

In [4]:
device = 'cuda'  # Pode ser 'cuda' ou 'cpu'

# Dataloader artificial só para ilustração
target = torch.zeros(16, dtype=torch.long)
dl = [(x,target)]*10
optim = torch.optim.SGD(model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

for imgs, targets in dl:
    imgs = imgs.to(device)
    targets = targets.to(device)
    model.zero_grad()
    scores = model(imgs)
    loss = loss_func(scores, targets)
    loss.backward()
    optim.step()

    # Para logar a loss, podemos usar .item(), o que copia o valor de volta para a CPU
    loss_log = loss.item()
    # Podemos também usar .detach(), o que mantém o valor na GPU e evita uma cópia
    #loss_log = loss.detach()

### Programação assíncrona

É importante notar que a execução na CPU e na GPU é feita de forma assíncrona, isto é, enquanto a GPU está processando os dados, o programa continua executando na CPU. Vamos entender isso na prática.

In [5]:
import time
import torch

# 100 matrizes de tamanho 900 x 900
x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    # Multiplicação de 100 matrizes
    y = torch.matmul(x, x)
tt = time.perf_counter() - ti
print(tt)

0.026443400000061956


O código acima executa extremamente rápido! 

...será? Veja o que acontece se repetirmos exatamente o mesmo código mas mandarmos imprimir um valor do resultado:

In [6]:
import time
import torch

x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    y = torch.matmul(x, x)
# Tempo de execução do loop
t_loop = time.perf_counter() - ti
print(y[0,0,0])
# Tempo de execução do loop + print
t_print = time.perf_counter() - ti
print(t_loop, t_print)

tensor(9.5057, device='cuda:0')
0.023584899999150366 10.395847499999945


O tempo medido após o print é muito maior do que o medido logo antes do print! Por acaso o print demorou para executar? Não, o que aconteceu é que a impressão de um valor de `y` é uma tarefa que bloqueia a CPU. O processo fica esperando a GPU terminar os cálculos para poder imprimir o valor.

O fato da execução na CPU e GPU serem assíncronas possui duas implicações importantes:

* Para garantir a máxima performance, é preciso tomar cuidado com operações que bloqueiam a CPU. Cópias entre a CPU e GPU bloqueiam, assim como impressão de valores na tela.
* É preciso tomar cuidado ao medir o tempo de execução de um código. O tempo é medido no processo da CPU, e não necessariamente leva em conta o tempo de execução na GPU

Copiar o resultado da GPU para a CPU bloqueia:

In [7]:
x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    y = torch.matmul(x, x)
    loss = y.sum()
    loss = loss.to('cpu')
    # O mesmo ocorreria com os comandos
    #loss = loss.item()
    #print(loss)
tt = time.perf_counter() - ti
print(tt)

5.440150800000083


### Medindo performance

Vamos ver técnicas simples para medir o tempo de execução na CPU e GPU e o uso de memória na GPU

In [14]:
import torch
import time

def proc():
    '''Processamento a ser executado'''
    x = torch.randn(100, 900, 900, device='cuda')
    for i in range(500):
        y = torch.matmul(x, x)

def benchmark(func):

    # Eventos de medida de tempo na GPU
    gpu_start = torch.cuda.Event(enable_timing=True)
    gpu_end = torch.cuda.Event(enable_timing=True)  
    # Apaga registro de pico de memória
    torch.cuda.reset_peak_memory_stats()

    # Tempo inicial na CPU
    ti = time.perf_counter()
    # Envia um comando para a GPU para registrar o tempo
    gpu_start.record() 
    func()
    # Tempo final na CPU
    t_cpu = time.perf_counter() - ti
    # Tempo final na GPU
    gpu_end.record()
    # Espera a GPU terminar os cálculos
    torch.cuda.synchronize()

    t_gpu = gpu_start.elapsed_time(gpu_end)/1000
    max_memory = torch.cuda.max_memory_allocated()/2**30

    return t_cpu, t_gpu, max_memory

t1, t2, m = benchmark(proc)
print(f'{t1:.3f}s, {t2:.3f}s, {m:.1f}GiB')

0.017s, 5.505s, 1.6GiB
